In [67]:
import numpy as np
import pandas as pd
import os
import statistics
from scipy.interpolate import CubicSpline
from datetime import datetime
import matplotlib.pyplot as plt
from gtda.time_series import SingleTakensEmbedding, takens_embedding_optimal_parameters
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud
from gtda.diagrams import PersistenceEntropy, PairwiseDistance
from gtda.diagrams import Silhouette
from gtda.diagrams import BettiCurve 
from gtda.homology import VietorisRipsPersistence
from gtda.time_series import TakensEmbedding, SlidingWindow
import plotly.express as px
from sklearn.metrics import mutual_info_score
from gtda.metaestimators import CollectionTransformer
from gtda.pipeline import Pipeline
from gtda.plotting import plot_heatmap

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [222]:
homology_dimensions = (0,1)
VRP = VietorisRipsPersistence(homology_dimensions=homology_dimensions)
PE = PersistenceEntropy()
PE_norm = PersistenceEntropy(normalize=True)
Betti = BettiCurve()

window_size = 128*4  # 128 points at 6.4 kHz is a window of 0.02 second
window_stride = int(window_size*0.8)  # 20% of each window is overlapping with the previous one

def compute_max_persistence(sliding_window):
    pers_H1 = []
    pers_H0 = []
    max_pers_H0 = []
    max_pers_H1 = []

    for PersDiag in sliding_window: 
        for point in PersDiag:
            birth = point[0]
            death = point[1]
            dimension = point[2]
            persistence = abs(death - birth)
            pers_H0.extend([persistence] if dimension == 0 else [])
            pers_H1.extend([persistence] if dimension == 1 else [])
        max_pers_H0.append(np.amax(pers_H0))
        max_pers_H1.append(np.amax(pers_H1))
        pers_H0 = []
        pers_H1 = []

    return statistics.mean(max_pers_H0), statistics.mean(max_pers_H1), statistics.stdev(max_pers_H0), statistics.stdev(max_pers_H1)

def compute_persistence_entropy(ts_entropy):

    Entropy_H0_norm = []
    Entropy_H1_norm = []

    for item in ts_entropy:
        Entropy_H0_norm.append(item[0])
        Entropy_H1_norm.append(item[1])
    
    return statistics.mean(Entropy_H0_norm), statistics.mean(Entropy_H1_norm), statistics.stdev(Entropy_H0_norm), statistics.stdev(Entropy_H1_norm)



def compute_betti_curves(ts_betti):

    meanBettiH0 = []
    meanBettiH1 = []

    for i in range(len(ts_betti)):
        meanBettiH0.append(statistics.mean(ts_betti[i,0,:]))
    for i in range(len(ts_betti)):
        meanBettiH1.append(statistics.mean(ts_betti[i,1,:]))

    return statistics.mean(meanBettiH0), statistics.stdev(meanBettiH0), statistics.mean(meanBettiH1), statistics.stdev(meanBettiH1)

    


In [228]:
def resample_signal(df, frequency, signal_cutoff):
    # ====================================================================================================
    # this functions resamples the signal to the desired frequency and cuts it to the desired length
    # ====================================================================================================

    # check the data time resolution of the data
    current_resolution_seconds = df['t'][1] - df['t'][0]
    current_resolution_kHz = 1.0 / (1000 * current_resolution_seconds)
    
    if current_resolution_kHz > frequency+0.5:
        print('Signal frequency:', current_resolution_kHz,'Resampling the signal to the expected frequency of', frequency, 'kHz')
    
        #resample the signal    
        if current_resolution_kHz / frequency == 4: # only for the case of 6.4 kHz to 25.4 kHz
            df_resampled = df.iloc[::4, :]
        else:
            print('Error, the signal frequency is not a multiple of the expected frequency of', frequency, 'kHz')
            exit()
    elif current_resolution_kHz <= frequency+0.5 and current_resolution_kHz >= frequency-0.5:
        df_resampled = df
    else:
        print('ERROR, the signal frequency is lower than the expected frequency of', frequency, 'kHz')
        exit()

    # cut the signal to cutoff
    signal_length = df_resampled['t'].iloc[-1] - df_resampled['t'].iloc[0]

    # resetting the index
    df_resampled = df_resampled.reset_index(drop=True)

    if signal_length > signal_cutoff:
        # Find the index closest to 'cutoff' seconds
        closest_index = df_resampled['t'].sub(signal_cutoff).abs().idxmin()

        # Slice the DataFrame at the closest index
        df_resampled = df_resampled.iloc[:closest_index + 1]
        
    return df_resampled


def normalize_signal(df, var):

    df[var] = df[var] - df[var].mean() # remove mean
    df[var] = df[var] / df[var].std()  # normalize
    
    return df

In [225]:
def run_tda_pipeline(variable, df):
    # ====================================================================================================
    # this functions interpolates the signal (for smoother results) runs the TDA pipeline  
    # ====================================================================================================

    signal = df[variable].values

    print("Running TDA pipeline for variable: ", variable, signal.shape)

    #1 interpolate
    xmin = 1
    #xmax = len(signal)-1
    xmax = int(len(signal))-1 # taking only the first half of the signal
    some_step = 0.25                         
    cs = CubicSpline(range(len(signal)), signal)
    x_range = np.arange(xmin, xmax, some_step)

    #2 sliding windows
    SW = SlidingWindow(size=window_size, stride=window_stride)
    sliding_window = SW.fit_transform(cs(x_range)) # fit_transform is necessary to get the windows  
    
    #3 estimate parameters for takens embedding (fixed for now)
    time_delay = 12
    embedding_dimension = 9

    #4 takens embedding
    TE = TakensEmbedding(time_delay=time_delay, dimension=embedding_dimension, stride=1)
    ts_embed  = TE.fit_transform(sliding_window)

    #5 persistent homology        
    ts_diagrams = VRP.fit_transform(ts_embed)
    #ts_entropy = PE.fit_transform(ts_diagrams)
    ts_entropynorm = PE_norm.fit_transform(ts_diagrams)
    ts_Betti = Betti.fit_transform(ts_diagrams) 

    #6 collect results
    avg_PH0, avg_PH1, std_PH0, std_PH1 = compute_max_persistence(ts_diagrams)
    avg_EH0, avg_EH1, std_EH0, std_EH1 = compute_persistence_entropy(ts_entropynorm)
    avg_BH0, avg_BH1, std_BH0, std_BH1 = compute_betti_curves(ts_Betti)

    #7 return
    results = [avg_PH0, std_PH0, avg_PH1, std_PH1, avg_EH0, std_EH0, avg_EH1, std_EH1, avg_BH0, std_BH0, avg_BH1, std_BH1]

    return results

In [230]:
def analyze_data(df, var, frequency, norm=False):
    # ====================================================================================================
    # this functions triggers resamples the signal to the desired frequency and cuts it to the desired length
    # ====================================================================================================

    df = resample_signal(df, frequency, 5) # I am adding manually a cutoff of 5 seconds to the signal to analyze to save time

    if norm :
        print("Normalizing signal")
        normalize_signal(df, var)
    
    results = run_tda_pipeline(var, df)

    return results

In [165]:
def process_data(folder_path, variable, freq):
    # ====================================================================================================
    # this functions reads the data from the folder, fixes timestamps and triggers the analysis
    # ====================================================================================================

    output_format = "%Y-%m-%d %H:%M:%S"
    #output_format = "%Y%m%d%H%M%S"    
    input_format = "WTG64-%Y-%m-%dT%H_%M_%SZ"
    results_array = []
    timing_array = []

    # Reading the data from the folder

    for parquet_file in os.listdir(folder_path):
        if parquet_file.endswith('vib.parquet'):
            file_name_parts = parquet_file.split('-')[1:-1]
            name = '-'.join(file_name_parts)

            # Parse the input string to a datetime object
            timestamp = datetime.strptime(name, input_format)

            # Convert the datetime object to a string with the desired format
            formatted_timestamp = timestamp.strftime(output_format)

            # Read the vib.parquet file into a DataFrame
            df = pd.read_parquet(os.path.join(folder_path, parquet_file))
            
            # Run the analysis
            results = analyze_data(df, variable, freq, False)
            
            timing_array.append(formatted_timestamp)
            results_array.append(results)

    return results_array, timing_array

In [126]:
def save_and_plot(variable_name, output_file_path, results_array, timestamps_array):

    column_names = [
        "Timestamp",
        "MaxPersH0", "StdPersH0",
        "MaxPersH1", "StdPersH1",
        "AvgEntropyH0", "StdEntropyH0",
        "AvgEntropyH1", "StdEntropyH1",
        "AvgBettiH0", "StdBettiH0",
        "AvgBettiH1", "StdBettiH1"
    ]

    # Fill missing values (from files with different sampling rate) with NaN
    results_array = [row if row != [] else [np.nan] * 12 for row in results_array]
    print("Results array: ", len(results_array))
    
    # Write results to file
    with open(output_file_path, 'w') as file:
        # Write header
        file.write(" # Timestamp, MaxPersH0, StdPersH0, MaxPersH1, StdPersH1, AvgEntropyH0, StdEntropyH0, AvgEntropyH1, StdEntropyH1, AvgBettiH0, StdBettiH0, AvgBettiH1, StdBettiH1\n")

        for timestamp, row in zip(timestamps_array, results_array):
            if row is None:
                continue
            data_row = ' '.join([str(timestamp)] + [str(item) for item in row]) + '\n'
            file.write(data_row)

    print("Text file '{}' created.".format(output_file_path))

    # Create a DataFrame using a dictionary
    data = {'Timestamp': timestamps_array}
    for i, col_name in enumerate(column_names[1:]):
        data[col_name] = [row[i] for row in results_array]

    df = pd.DataFrame(data)
    df = df.sort_values(by='Timestamp', ascending=True)
    df.to_csv(variable_name + '.csv', index=False)  # Save DataFrame as CSV

    print("DataFrame '{}' created and saved.".format(variable_name))

    return df 


In [127]:
def plot_data(df, plot_name):

    fig = make_subplots(rows=3, cols=1)

    fig.add_trace(go.Scatter(
        x=df['Timestamp'],
        y=df['MaxPersH0'],
        mode='lines+markers',  # Add markers for data points
        error_y=dict(
            type='data',  # error type: data-based error
            array=df['StdPersH0'],  # array of error values
            visible=True
        ),
        name="MaxPersH0"
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df['Timestamp'],
        y=df['MaxPersH1'],
        mode='lines+markers',  # Add markers for data points
        error_y=dict(
            type='data',  # error type: data-based error
            array=df['StdPersH1'],  # array of error values
            visible=True
        ),
        name="MaxPersH1"
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=df['Timestamp'],
        y=df['AvgEntropyH1'],
        mode='lines+markers',  # Add markers for data points
        error_y=dict(
            type='data',
            array=df['StdEntropyH1'],
            visible=True
        ),
        name="AvgEntropyH1"
    ), row=3, col=1)

    fig.update_layout(height=600, width=1200, title_text=plot_name)

    fig.show()
    return

In [231]:
results, timestamps = process_data('./', 'GbxIss', 6.4)

Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Running TDA pipeline for variable:  GbxIss (32001,)
Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Signal frequency: 25.6 Resampling the signal to the expected frequency of 6.4 kHz
Running TDA pipeline for variable:  GbxIss (32001,)
Signal fre

In [232]:
df_GbxIss_resampled_6kHz = save_and_plot('GbxIss', 'GbxIss_6kHz_resampled.dat', results, timestamps)

Results array:  50
Text file 'GbxIss_6kHz_resampled.dat' created.
DataFrame 'GbxIss' created and saved.


In [233]:
plot_data(df_GbxIss_resampled_6kHz, 'All data resampled to 6.4 kHz')

Changing the resolution input, to analyze only the 6 kHz signals.

In [96]:
results, timestamps = process_data('./', 'GbxIss')
df_GbxIss_6kHz = save_and_plot('GbxIss', 'GbxIss_6kHz.dat', results, timestamps)
plot_data(df_GbxIss_6kHz, 'df_GbxIss_6kHz')

Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (201, 512)
Results array:  50
Text file 'GbxIss_6kHz.dat' created.
DataFrame 'GbxIss' created and saved.


In [107]:
results, timestamps = process_data('./', 'GbxIss')
df_GbxIss_25kHz = save_and_plot('GbxIss', 'GbxIss_25kHz.dat', results, timestamps)

Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss (262144,)
Sliding window shape:  (407, 512)
Running TDA pipeline for variable:  GbxIss 

In [108]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(
    x=df_GbxIss_25kHz['Timestamp'],
    y=df_GbxIss_25kHz['MaxPersH0'],
    mode='lines+markers',  # Add markers for data points
    error_y=dict(
        type='data',  # error type: data-based error
        array=df_GbxIss_25kHz['StdPersH0'],  # array of error values
        visible=True
    ),
    name="MaxPersH0 25kHz"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_GbxIss_6kHz['Timestamp'],
    y=df_GbxIss_6kHz['MaxPersH0'],
    mode='lines+markers',  # Add markers for data points
    error_y=dict(
        type='data',  # error type: data-based error
        array=df_GbxIss_6kHz['StdPersH0'],  # array of error values
        visible=True
    ),
    name="MaxPersH0 6kHz"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_GbxIss_25kHz['Timestamp'],
    y=df_GbxIss_25kHz['MaxPersH1'],
    mode='lines+markers',  # Add markers for data points
    error_y=dict(
        type='data',  # error type: data-based error
        array=df_GbxIss_25kHz['StdPersH1'],  # array of error values
        visible=True
    ),
    name="MaxPersH1 25kHz"
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_GbxIss_6kHz['Timestamp'],
    y=df_GbxIss_6kHz['MaxPersH1'],
    mode='lines+markers',  # Add markers for data points
    error_y=dict(
        type='data',  # error type: data-based error
        array=df_GbxIss_6kHz['StdPersH1'],  # array of error values
        visible=True
    ),
    name="MaxPersH1 6kHz"
), row=2, col=1)

fig.update_layout(height=600, width=1200, title_text="plot_name")

fig.show()
